In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import numpy as np
from kronfluence.analyzer import Analyzer, prepare_model
from kronfluence.task import Task
from kronfluence.arguments import FactorArguments

In [2]:
# Define ResNet-9
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [3]:
class ResNet9(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet9, self).__init__()
        self.prep = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.layer1 = nn.Sequential(
            BasicBlock(64, 128, stride=2),
            BasicBlock(128, 128, stride=1)
        )

        self.layer2 = nn.Sequential(
            BasicBlock(128, 256, stride=2),
            BasicBlock(256, 256, stride=1)
        )

        self.layer3 = nn.Sequential(
            BasicBlock(256, 512, stride=2),
            BasicBlock(512, 512, stride=1)
        )

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.prep(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [5]:
# Define transformation and load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

In [6]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:03<00:00, 46300759.56it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
# Introduce label swaps in the training set
num_swaps = 500
swap_indices = np.random.choice(len(train_dataset), num_swaps, replace=False)
for idx in swap_indices:
    current_label = train_dataset.targets[idx]
    new_label = (current_label + 1) % 10  # Change to the next label
    train_dataset.targets[idx] = new_label

In [ ]:
# Define the training task for kronfluence
class ClassificationTask(Task):
    def compute_train_loss(self, batch, model, sample=False):
        inputs, labels = batch
        logits = model(inputs)
        if not sample:
            return nn.functional.cross_entropy(logits, labels, reduction='sum')
        with torch.no_grad():
            probs = nn.functional.softmax(logits, dim=-1)
            sampled_labels = torch.multinomial(probs, num_samples=1).flatten()
        return nn.functional.cross_entropy(logits, sampled_labels.detach(), reduction='sum')

    def compute_measurement(self, batch, model):
        inputs, labels = batch
        logits = model(inputs)
        bindex = torch.arange(logits.shape[0]).to(device=logits.device, non_blocking=False)
        logits_correct = logits[bindex, labels]
        cloned_logits = logits.clone()
        cloned_logits[bindex, labels] = torch.tensor(-torch.inf, device=logits.device, dtype=logits.dtype)
        margins = logits_correct - cloned_logits.logsumexp(dim=-1)
        return -margins.sum()

In [27]:
with open('manip_idx.txt', 'r') as file:
    line = file.readline().strip()
    manip_idx = [item.strip() for item in line.split(',')]
print(manip_idx)

['18212', '45059', '13554', '35035', '38863', '37496', '44475', '39787', '43845', '23321', '44420', '35072', '17268', '9598', '19970', '31211', '4064', '48667', '41680', '31325', '4246', '33942', '18596', '23725', '49024', '43947', '17870', '37745', '47244', '4229', '25523', '28305', '1903', '8757', '18213', '40212', '21962', '30920', '38729', '33333', '36418', '8431', '29878', '4190', '6326', '12891', '23495', '8004', '12976', '38878', '40615', '47340', '19284', '10370', '40192', '5846', '3816', '17828', '48681', '7419', '15404', '47405', '19703', '45774', '47555', '6560', '16979', '31956', '43800', '14659', '33545', '34657', '32199', '31749', '33918', '31397', '28454', '18976', '969', '35628', '33957', '220', '34605', '6839', '32322', '15077', '8162', '33427', '5815', '21756', '28668', '32492', '27064', '42280', '34391', '537', '47965', '7330', '17854', '20750', '43624', '36022', '383', '4060', '7174', '14976', '3482', '8866', '28877', '19591', '39275', '11911', '36489', '36255', '37

In [28]:
with open('harmful_idx.txt', 'r') as file:
    line = file.readline().strip()
    detected_idx = [item.strip() for item in line.split(',')]
print(detected_idx)
print(len(detected_idx))

['1', '2', '3', '6', '9', '10', '11', '12', '15', '16', '21', '27', '32', '38', '43', '45', '46', '49', '50', '51', '52', '53', '56', '61', '62', '68', '70', '73', '74', '83', '86', '87', '94', '101', '102', '104', '106', '108', '111', '112', '113', '114', '117', '118', '119', '124', '127', '130', '132', '134', '135', '137', '139', '146', '148', '150', '152', '153', '154', '155', '156', '157', '166', '174', '177', '180', '181', '186', '187', '191', '195', '203', '208', '214', '219', '230', '232', '233', '235', '237', '239', '241', '244', '245', '246', '247', '250', '254', '255', '257', '258', '263', '266', '267', '270', '271', '272', '274', '278', '283', '285', '289', '291', '293', '295', '297', '299', '302', '304', '305', '306', '307', '310', '311', '313', '316', '317', '324', '325', '327', '330', '335', '337', '338', '339', '346', '350', '357', '367', '371', '373', '374', '377', '380', '383', '385', '394', '395', '401', '406', '409', '433', '434', '443', '445', '447', '449', '450', '

In [29]:
detected = 0
for idx in manip_idx:
    if idx in detected_idx:
        detected += 1
print(detected)

1676
